In [4]:


class ProgBar:

    def __init__(self, n_elements,int_str):
        
        import sys

        self.n_elements = n_elements
        self.progress = 0

        print(int_str)

        # initiallizing progress bar

        info = '{:.2f}% - {:d} of {:d}'.format(0,0,n_elements)

        formated_bar = ' '*int(50)

        sys.stdout.write("\r")

        sys.stdout.write('[%s] %s' % (formated_bar,info))

        sys.stdout.flush()

    def update(self,prog_info=None):
        
        import sys

        if prog_info == None:

            self.progress += 1

            percent = (self.progress)/self.n_elements * 100 / 2

            info = '{:.2f}% - {:d} of {:d}'.format(percent*2,self.progress,self.n_elements)

            formated_bar = '-'* int (percent) + ' '*int(50-percent)

            sys.stdout.write("\r")

            sys.stdout.write('[%s] %s' % (formated_bar,info))

            sys.stdout.flush()


        else:

            self.progress += 1

            percent = (self.progress)/self.n_elements * 100 / 2

            info = '{:.2f}% - {:d} of {:d} '.format(percent*2,self.progress,self.n_elements) + prog_info

            formated_bar = '-'* int (percent) + ' '*int(50-percent)

            sys.stdout.write("\r")

            sys.stdout.write('[%s] %s' % (formated_bar,info))

            sys.stdout.flush()


class PDF_miner:

    def __init__(self,folder):

        import PyPDF2

        self.folder = folder

        reader = PyPDF2.PdfFileReader(folder + '/slide.pdf')

        self.reader = PyPDF2.PdfFileReader(folder + '/slide.pdf')

        self.pdf_n_pages = reader.numPages

        self.pic_ref_list = []

        self.discart_list = []


    def convert_pdf_to_string(self):

        from io import StringIO
        from pdfminer.converter import TextConverter
        from pdfminer.layout import LAParams
        from pdfminer.pdfdocument import PDFDocument
        from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
        from pdfminer.pdfpage import PDFPage
        from pdfminer.pdfparser import PDFParser


        output_string = StringIO()
        with open('transition.pdf', 'rb') as in_file:
            parser = PDFParser(in_file)
            doc = PDFDocument(parser)
            rsrcmgr = PDFResourceManager()
            device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
            interpreter = PDFPageInterpreter(rsrcmgr, device)
            for page in PDFPage.create_pages(doc):
                interpreter.process_page(page)

        return(output_string.getvalue())


    def add_line(self,line,out):

        # Open the file in append & read mode ('a+')

        with open(self.folder+out, "a+") as file_object:

            # Move read cursor to the start of file.
            file_object.seek(0)

            # If file is not empty then append '\n'
            data = file_object.read(100)

            if len(data) > 0 :
                file_object.write("\n")
                
            # Append text at the end of file
            file_object.write(line)

    
    def get_text(self,pg):
        
        import PyPDF2

        writer = PyPDF2.PdfFileWriter()

        output_filename = 'transition.pdf'

        page = self.reader.getPage(pg)

        writer.addPage(page)

        with open(output_filename, 'wb') as output:
            writer.write(output)

        del writer

        text = self.convert_pdf_to_string()

        text = text.replace('\x0c',' ')
        text = text.replace('%','\%')
        text = text.split('\n')
        text = text[5:]

        return text


    def create_fig_form(self,type="fig"):

        for i in range(1,self.pdf_n_pages+1):

            if type == "fig": name = "/fig_form.txt"

            else: name = "/eq_form.txt"

            self.add_line("{}:0".format(i),name)


    def extract_figures(self):

        import fitz
        import glob

        # opening pdf file

        self.doc = fitz.open(self.folder + "/slide.pdf")

        figures_list = glob.glob(self.folder + '/figures/*')

        for pg in range(self.pdf_n_pages):

            for img in self.doc.get_page_images(pg):
                
                # get figure refferance
                
                xref = img[0]
                
                # build figure bitmap

                pix = fitz.Pixmap(self.doc, xref)

                # defines the name of the picture

                file_name = self.folder + ("/figures/%s.png" % (xref))

                # if the figure does not exists save it

                if not file_name in figures_list:

                    if pix.n < 5:       # this is GRAY or RGB
                        pix.save(file_name)
                    else:               # CMYK: convert to RGB first
                        pix1 = fitz.Pixmap(fitz.csRGB, pix)
                        pix1.save(file_name)
                        pix1 = None
                    pix = None

        print('Figure Extraction done!')

        print('='*22)

        self.create_fig_form()

        print("\nFill the figures location form and press ENTER")

        input("Fill th figures location form and press ENTER")

        with open(self.folder + "/fig_form.txt", "r+") as file_object:

            fig_loc = file_object.read()

        fig_loc = fig_loc.replace(' ','')
        fig_loc = fig_loc.split('\n')

        self.fig_loc = fig_loc


    def extract_equations(self):

        import numpy as np
        import shutil
        import glob

        print("\nPress ENTER to confirm that the equation prints are in the correct folder")

        input("Press ENTER to confirm that the equation prints are in the correct folder")

        figures_list = glob.glob(self.folder + '/equations/*')

        figures_list = np.sort(figures_list)

        for i,fig in enumerate(figures_list):

            file_name = self.folder + '/equations/' + str(i+1) + '.png'

            shutil.move(fig, file_name)

        self.create_fig_form("eq")

        print("\nFill the equations location form and press ENTER")

        input("Fill the equations location form and press ENTER")

        with open(self.folder + "/eq_form.txt", "r+") as file_object:

            fig_loc = file_object.read()

        fig_loc = fig_loc.replace(' ','')
        fig_loc = fig_loc.split('\n')

        self.eq_loc = fig_loc


    def get_figures (self,pg):

        pg_figs = self.fig_loc[pg]

        pg_figs = pg_figs.split(':')[1:]

        pg_figs = pg_figs[0].split(',')

        fig_call_list = []

        for fig in pg_figs:

            if not fig == '0':

                figure_call = [
                    '\\begin{figure}[h!]',
                    '\centering',
                    ('\includegraphics[scale=0.5]{Pictures/' + fig + '.png}'),
                    '\caption{Caption}',
                    '\label{fig:my_label}',
                    '\end{figure}',
                    '',
                    ''
                ]

                for i in range(len(figure_call)): fig_call_list.append(figure_call[i])


        return fig_call_list


    def get_equations(self,pg):

        pg_eq = self.eq_loc[pg]

        pg_eq = pg_eq.split(':')[1:]

        pg_eq = pg_eq[0].split(',')

        fig_call_list = []

        for fig in pg_eq:

            if not fig == '0':

                figure_call = [
                    '\\begin{figure}[h!]',
                    '\centering',
                    ('\includegraphics[scale=0.8]{Equations/' + fig + '.png}'),
                    '\caption*{}',
                    '\label{fig:my_label}',
                    '\end{figure}',
                    '',
                    ''
                ]

                for i in range(len(figure_call)): fig_call_list.append(figure_call[i])


        return fig_call_list


    def convert(self):

        self.pages_struct = {}

        self.extract_figures()

        self.extract_equations()

        # Initializing progress bar

        bar = ProgBar(self.pdf_n_pages,'\nConverting PDF to LaTex...')

        for pg in range(self.pdf_n_pages):

            self.pages_struct[pg] = {}

            self.pages_struct[pg]['text'] = self.get_text(pg)

            self.pages_struct[pg]['equations'] = self.get_equations(pg)

            self.pages_struct[pg]['figures'] = self.get_figures(pg)

            bar.update()


    def save(self):

        ### Saving results in a txt file 

        # initializing progress bar

        bar = ProgBar(len(self.pages_struct),"\n\nSalving conversion...")

        for pg in self.pages_struct:
            
            # appending text

            for txt in self.pages_struct[pg]['text']:
                self.add_line(txt,"/conv.txt")

            # appending equations

            for fig in self.pages_struct[pg]['equations']:
                self.add_line(fig,"/conv.txt")

            # appending figures

            for fig in self.pages_struct[pg]['figures']:
                self.add_line(fig,"/conv.txt")

            # adding new page

            self.add_line('\\newpage',"/conv.txt")

            bar.update()
        

In [5]:
# initializing class

converter = PDF_miner('modulo_7')

# converting information

converter.convert()

# saving results


converter.save()

Figure Extraction done!

Fill the figures location form and press ENTER

Press ENTER to confirm that the equation prints are in the correct folder

Fill the equations location form and press ENTER

Converting PDF to LaTex...
[--------------------------------------------------] 100.00% - 65 of 65

Salving conversion...
[--------------------------------------------------] 100.00% - 65 of 65

In [114]:
import fitz

# format figure call

doc = fitz.open("Capitulo_7/slide.pdf")

pg_fig_list = []

for i in range(len(doc)):
    
    blocks = doc[i].get_text('blocks')

    for bck in blocks:
        bck_type = bck[4].split(':')[0]
        if bck_type == '<image':
            print(bck)
            pic_ref = bck[5] + 1
            file_name = str(pic_ref) + ".png"

            pg_fig_list.append(pic_ref)

(19.799999237060547, 0.07169342041015625, 195.6999969482422, 83.17169189453125, '<image: DeviceRGB, width: 254, height: 120, bpc: 8>', 3, 1)
(552.7999877929688, 26.371681213378906, 720.0, 79.9716796875, '<image: DeviceRGB, width: 240, height: 77, bpc: 8>', 4, 1)
(19.799999237060547, 0.07169342041015625, 195.6999969482422, 83.17169189453125, '<image: DeviceRGB, width: 254, height: 120, bpc: 8>', 3, 1)
(552.7999877929688, 26.371681213378906, 720.0, 79.9716796875, '<image: DeviceRGB, width: 240, height: 77, bpc: 8>', 4, 1)
(19.799999237060547, 0.07169342041015625, 195.6999969482422, 83.17169189453125, '<image: DeviceRGB, width: 254, height: 120, bpc: 8>', 3, 1)
(552.7999877929688, 26.371681213378906, 720.0, 79.9716796875, '<image: DeviceRGB, width: 240, height: 77, bpc: 8>', 4, 1)
(493.8999938964844, 186.9716796875, 908.4000244140625, 325.8716735839844, '<image: DeviceRGB, width: 624, height: 209, bpc: 8>', 11, 1)
(206.0, 368.3717041015625, 674.0, 525.1716918945312, '<image: DeviceRGB, wi

In [113]:
pg_fig_list = np.unique(pg_fig_list)

pg_fig_list

array([ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13])

In [ ]:
### getting info from pages

pages_struct = {}

folder = 'Capitulo_7'

reader = PyPDF2.PdfFileReader(folder + '/slide.pdf')

pdf_n_pages = reader.numPages

writer = PyPDF2.PdfFileWriter()

### Reading text and saving in the dictionary

# Initializing progress bar

bar = ProgBar(pdf_n_pages,'Reading text and saving in the dictionary...')


for pg in range(pdf_n_pages):

    pages_struct[pg] = {
        'figures': [],
        'text': 0,
        'equations': []
    }

    pages_struct[pg]['text'] = get_text(pg)

    bar.update()


"""
# Reading figures and saving in the dictionary

figures_list  = glob.glob(folder+'/figures/*')

for i in range(len(figures_list)): 

    # extracting the page and format of the file's names

    name,form = figures_list[i].split('/')[-1].split('.')

    try:
    
        pg,_= name.split('-')

    except:

        pg = name

    figure_call = [
        '\\begin{figure}[h!]',
        '\centering',
        ('\includegraphics[scale=0.5]{Pictures/' + folder + '/figures/' + name + '.' + form + '}'),
        '\caption{Caption}',
        '\label{fig:my_label}',
        '\end{figure}'
        ''
        ''
    ]

    for i in range(len(figure_call)): pages_struct[int(pg)-1]['figures'].append(figure_call[i])

    """

In [ ]:
### Saving results in a txt file 

for pg in pages_struct:
    
    # appending text

    for txt in pages_struct[pg]['text']:
        add_line(folder,txt)

    # appending equations

    for eq in pages_struct[pg]['equations']:
        add_line(folder,eq)

    # appending figures

    for fig in pages_struct[pg]['figures']:
        add_line(folder,fig)

    # adding new page

    add_line(folder,'\\newpage')

In [16]:
import glob

file_list = glob.glob('Capitulo_7/figures/*')

a = 'Capitulo_7/figures/4.png'

if not a in file_list:

    print(a in file_list)

False


In [13]:
##### Deprecation

def extract_figures (folder):

    import fitz
    
    # opening pdf file

    doc = fitz.open(folder + "/slide.pdf")

    # getting number of pages

    num_pages = doc.page_count

    # initializaing progress bar

    bar =  ProgBar(num_pages,'Extracting Figures...')

    for i in range (num_pages):
        for img in enumerate(doc.get_page_images(i)):
            xref = img[0]
            print(img)
            pix = fitz.Pixmap(doc, xref)
            if pix.n < 5:       # this is GRAY or RGB
                pix.save(folder + "/figures/%s.png" % (xref))
            else:               # CMYK: convert to RGB first
                pix1 = fitz.Pixmap(fitz.csRGB, pix)
                pix1.save(folder + "/figures/%s.png" % (xref))
                pix1 = None
            pix = None



In [34]:
extract_figures('Capitulo_7')

Extracting Figures...
[--------------------------------------------------] 100.00% - 69 of 69

Figure Extraction done!

Which should be discarted?: 
['4', '5', '99']


In [65]:
import fitz

doc = fitz.open("Capitulo_7/slide.pdf")

fig_call_list = []

pg_fig_list = []

blocks = doc[0].get_text('blocks')

for bck in blocks:
    bck_type = bck[4].split(':')[0]
    if bck_type == '<image':
        pic_ref = bck[5] + 1
        file_name = "Capitulo_7/figures/" + str(pic_ref) + ".png"

        pg_fig_list.append(file_name)


for file in pg_fig_list:
            
    figure_call = [
        '\\begin{figure}[h!]',
        '\centering',
        ('\includegraphics[scale=0.5]{Pictures/' + file + '}'),
        '\caption{Caption}',
        '\label{fig:my_label}',
        '\end{figure}',
        '',
        ''
    ]

    for i in range(len(figure_call)): fig_call_list.append(figure_call[i])



['\\begin{figure}[h!]', '\\centering', '\\includegraphics[scale=0.5]{Pictures/Capitulo_7/figures/4.png}', '\\caption{Caption}', '\\label{fig:my_label}', '\\end{figure}', '', '', '\\begin{figure}[h!]', '\\centering', '\\includegraphics[scale=0.5]{Pictures/Capitulo_7/figures/5.png}', '\\caption{Caption}', '\\label{fig:my_label}', '\\end{figure}', '', '']


In [ ]:

for i,img in enumerate(doc.get_page_images(0)):
    xref = img[0]
    print(img)
    pix = fitz.Pixmap(doc, xref)
    if pix.n < 5:       # this is GRAY or RGB
        pix.save(folder + "/figures/p%s-%s.png" % (i, xref))
    else:               # CMYK: convert to RGB first
        pix1 = fitz.Pixmap(fitz.csRGB, pix)
        pix1.save(folder + "/figures/p%s-%s.png" % (i, xref))
        pix1 = None
    pix = None


In [10]:
### getting info from pages

pages_struct = {}

folder = 'Capitulo_7'

reader = PyPDF2.PdfFileReader(folder + '/slide.pdf')

pdf_n_pages = reader.numPages

writer = PyPDF2.PdfFileWriter()

### Reading text and saving in the dictionary

# Initializing progress bar

bar = ProgBar(pdf_n_pages,'Reading text and saving in the dictionary...')


for pg in range(pdf_n_pages):

    pages_struct[pg] = {
        'figures': [],
        'text': 0,
        'equations': []
    }

    pages_struct[pg]['text'] = get_text(pg)

    bar.update()


# saving formated equations on the dictionary

# listing equation's figures

figures_list  = glob.glob(folder+'/equations/*')

# sorting list o files

page_list = []

# getting list of pages with equations

for i in range(len(figures_list)): 

    # extracting the page and format of the file's names

    name,form = figures_list[i].split('/')[-1].split('.')
    
    pg,_= name.split('-')

    # appending pages with eaqutions

    page_list.append(int(pg))

# sorting list of pages 

page_list = list(np.sort(page_list))

# creating dictionary for appending the list  of figures in each page

eq_dict = {}

for pg in page_list:

    eq_dict[pg] = []


# getting list of eqautions in each page

for i in range(len(figures_list)): 

    # extracting the page and format of the file's names

    name,form = figures_list[i].split('/')[-1].split('.')
    pg,num = name.split('-')

    # appending numbe of the equations in this page

    eq_dict[int(pg)].append(int(num))

    # sorting equations

    eq_dict[int(pg)] = list(np.sort(eq_dict[int(pg)]))

# setting count of equations

k = 1

for pg in eq_dict:

    for  num in eq_dict[pg]:
        
        # extracting the page and number from dictionary

        name = '-'.join((str(pg),str(num)))

        # writing LaTex syntax

        figure_call = [
            '\\begin{figure}[h!]',
            '\centering',
            ('\includegraphics[scale=0.5]{Pictures/' + folder + '/equations/' + name + '.png}'),
            ('\caption{Equação' + str(k) + '}'),
            ('\label{fig:eq' + str(k) + '}'),
            '\end{figure}',
            ' ',
            ' '
        ]

        # incremeanting count

        k += 1

        # appending formated text in the dicctionary

        for i in range(len(figure_call)): 
            pages_struct[int(pg)-1]['equations'].append(figure_call[i])



"""
# Reading figures and saving in the dictionary

figures_list  = glob.glob(folder+'/figures/*')

for i in range(len(figures_list)): 

    # extracting the page and format of the file's names

    name,form = figures_list[i].split('/')[-1].split('.')

    try:
    
        pg,_= name.split('-')

    except:

        pg = name

    figure_call = [
        '\\begin{figure}[h!]',
        '\centering',
        ('\includegraphics[scale=0.5]{Pictures/' + folder + '/figures/' + name + '.' + form + '}'),
        '\caption{Caption}',
        '\label{fig:my_label}',
        '\end{figure}'
        ''
        ''
    ]

    for i in range(len(figure_call)): pages_struct[int(pg)-1]['figures'].append(figure_call[i])

    """

Reading text and saving in the dictionary...
[--------------------------------------------------] 100.00% - 69 of 69

"\n# Reading figures and saving in the dictionary\n\nfigures_list  = glob.glob(folder+'/figures/*')\n\nfor i in range(len(figures_list)): \n\n    # extracting the page and format of the file's names\n\n    name,form = figures_list[i].split('/')[-1].split('.')\n\n    try:\n    \n        pg,_= name.split('-')\n\n    except:\n\n        pg = name\n\n    figure_call = [\n        '\\begin{figure}[h!]',\n        '\\centering',\n        ('\\includegraphics[scale=0.5]{Pictures/' + folder + '/figures/' + name + '.' + form + '}'),\n        '\\caption{Caption}',\n        '\\label{fig:my_label}',\n        '\\end{figure}'\n        ''\n        ''\n    ]\n\n    for i in range(len(figure_call)): pages_struct[int(pg)-1]['figures'].append(figure_call[i])\n\n    "

In [11]:
### Saving results in a txt file 

for pg in pages_struct:
    
    # appending text

    for txt in pages_struct[pg]['text']:
        add_line(folder,txt)

    # appending equations

    for eq in pages_struct[pg]['equations']:
        add_line(folder,eq)

    # appending figures

    for fig in pages_struct[pg]['figures']:
        add_line(folder,fig)

    # adding new page

    add_line(folder,'\\newpage')

In [18]:
import fitz
doc = fitz.open("Capitulo_7/slide.pdf")
doc

Document('Capitulo_7/slide.pdf')

In [19]:
len(doc)

70

In [25]:
for i in range(len(doc)):
    page = doc[i]
    figures_list = page.get_image_bbox()
    print(len(figures_list))


TypeError: get_image_bbox() missing 1 required positional argument: 'name'

In [26]:
for page in doc:
    pix = page.getPixmap()
    pix.writeImage("page-%i.png" % page.number)

FitzDeprecation: 'getPixmap' removed from class 'Page' after v1.19.0 - use 'get_pixmap'. [__init__.py:169]
FitzDeprecation: 'writeImage' removed from class 'Pixmap' after v1.19.0 - use 'save'. [__init__.py:169]


In [ ]:

for i in range(len(doc)):

    for j,img in enumerate(doc.get_page_images(i)):
    
        file_name = folder + '/figures/' + str(page_num) + '_' + str(j) + ".png"

        xref = img[0]
        pix = fitz.Pixmap(doc, xref)
        if pix.n < 5:       # this is GRAY or RGB
            pix.save(file_name)
        else:               # CMYK: convert to RGB first
            pix1 = fitz.Pixmap(fitz.csRGB, pix)
            pix1.save(file_name)
            pix1 = None
        pix = None

In [5]:
### Saving results in a txt file 

for pg in pages_struct:
    
    # appending text

    for txt in pages_struct[pg]['text']:
        add_line(folder,txt)

    # appending equations

    for eq in pages_struct[pg]['equations']:
        add_line(folder,eq)

    # appending figures

    for fig in pages_struct[pg]['figures']:
        add_line(folder,fig)

    # adding new page

    add_line(folder,'\\newpage')